<a href="https://colab.research.google.com/github/ldsAS/Tibame-AI-Learning/blob/main/Tibame20250617_%E4%BD%BF%E7%94%A8%E7%B7%9A%E6%80%A7%E5%9B%9E%E6%AD%B8%E6%A8%A1%E5%9E%8B%E9%87%9D%E5%B0%8D%E4%BC%81%E9%B5%9D%E9%AB%94%E9%87%8D%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
from google.cloud import bigquery

# 設置 Google Cloud 認證 # 此為示範，實際上應放在環境變數裡面(要隱藏起來)
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"/content/tibame-gad251-31-0613-c339499c7e0e.json"


# 初始化 BigQuery 客戶端
client = bigquery.Client()
print('connection done')


connection done


In [1]:
us_dataset_id = 'tibame_gad251_31_dataset_us' #定義us資料集
eu_dataset_id = 'tibame_gad251_31_dataset_eu' #定義eu資料集

In [6]:
# 定義 SQL 查詢以創建模型
query = f"""
CREATE OR REPLACE MODEL `{us_dataset_id}.penguins_model`
OPTIONS
  (model_type='linear_reg',
  input_label_cols=['body_mass_g']) AS
SELECT
  *
FROM
  `bigquery-public-data.ml_datasets.penguins`
WHERE
  body_mass_g IS NOT NULL;
"""

# 執行 SQL 查詢以創建模型
query_job = client.query(query)
query_job.result()  # 等待查詢完成

print("線性回歸 model import BigQuery ML")


線性回師 model import BigQuery ML


In [10]:
# 定義 SQL 查詢以使用模型進行評估
query = f"""
SELECT
  *
FROM
  ML.EVALUATE(MODEL `{us_dataset_id}.penguins_model`,
    (
    SELECT
      *
    FROM
      `bigquery-public-data.ml_datasets.penguins`
    WHERE
      body_mass_g IS NOT NULL));

"""

# 執行 SQL 查詢以進行評估
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

# 印出預測結果
for row in results:
    print(row)


Row((227.01223667447184, 81838.15989216758, 0.005070447373501302, 173.08081641661738, 0.8723772534253442, 0.8723772534253442), {'mean_absolute_error': 0, 'mean_squared_error': 1, 'mean_squared_log_error': 2, 'median_absolute_error': 3, 'r2_score': 4, 'explained_variance': 5})


In [11]:
# 取得 BigQuery 回傳的欄位名稱
field_names = results.schema

# 印出評估結果
for name in field_names:
    for field, value in zip(field_names, row):
        print(f"{field.name}: {value}")

mean_absolute_error: 227.01223667447184
mean_squared_error: 81838.15989216758
mean_squared_log_error: 0.005070447373501302
median_absolute_error: 173.08081641661738
r2_score: 0.8723772534253442
explained_variance: 0.8723772534253442
mean_absolute_error: 227.01223667447184
mean_squared_error: 81838.15989216758
mean_squared_log_error: 0.005070447373501302
median_absolute_error: 173.08081641661738
r2_score: 0.8723772534253442
explained_variance: 0.8723772534253442
mean_absolute_error: 227.01223667447184
mean_squared_error: 81838.15989216758
mean_squared_log_error: 0.005070447373501302
median_absolute_error: 173.08081641661738
r2_score: 0.8723772534253442
explained_variance: 0.8723772534253442
mean_absolute_error: 227.01223667447184
mean_squared_error: 81838.15989216758
mean_squared_log_error: 0.005070447373501302
median_absolute_error: 173.08081641661738
r2_score: 0.8723772534253442
explained_variance: 0.8723772534253442
mean_absolute_error: 227.01223667447184
mean_squared_error: 81838.15

In [14]:
# 定義 SQL 查詢以使用模型進行預測
query = f"""
SELECT
  *
FROM
  ML.PREDICT(MODEL `{us_dataset_id}.penguins_model`,
    (
    SELECT
      *
    FROM
      `bigquery-public-data.ml_datasets.penguins`
    WHERE island = 'Biscoe'));

"""

# 執行 SQL 查詢以進行預測
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

In [15]:
# 將每筆 row 解成 tuple (row.values() 是一整筆，row[i] 是值)
rows_data = [tuple(row.values()) for row in results]

# 取得欄位名稱
columns = [field.name for field in results.schema]

# 建立 DataFrame
import pandas as pd
df = pd.DataFrame(rows_data, columns=columns)

# 顯示前幾筆資料
print(df.head())

   predicted_body_mass_g                              species  island  \
0            4681.782896    Gentoo penguin (Pygoscelis papua)  Biscoe   
1            3875.224470  Adelie Penguin (Pygoscelis adeliae)  Biscoe   
2            3303.096891  Adelie Penguin (Pygoscelis adeliae)  Biscoe   
3            3976.529009  Adelie Penguin (Pygoscelis adeliae)  Biscoe   
4            3457.923587  Adelie Penguin (Pygoscelis adeliae)  Biscoe   

   culmen_length_mm  culmen_depth_mm  flipper_length_mm  body_mass_g     sex  
0               NaN              NaN                NaN          NaN    None  
1              39.7             18.9              184.0       3550.0    MALE  
2              36.4             17.1              184.0       2850.0  FEMALE  
3              41.6             18.0              192.0       3950.0    MALE  
4              35.0             17.9              192.0       3725.0  FEMALE  


In [23]:
# 定義 SQL 查詢以使用模型進行解釋
query = f"""
SELECT
  *
FROM
  ML.EXPLAIN_PREDICT(MODEL `{us_dataset_id}.penguins_model`,
    (
    SELECT
      *
    FROM
      `bigquery-public-data.ml_datasets.penguins`
    WHERE island = 'Biscoe'),
    STRUCT(3 as top_k_features));

"""

# 執行 SQL 查詢以進行解釋
query_job = client.query(query)
results = query_job.result()  # 等待查詢完成

In [24]:
# 將每筆 row 解成 tuple (row.values() 是一整筆，row[i] 是值)
rows_data = [tuple(row.values()) for row in results]

# 取得欄位名稱
columns = [field.name for field in results.schema]

# 建立 DataFrame
import pandas as pd
df = pd.DataFrame(rows_data, columns=columns)

# 顯示前幾筆資料
print(df.head())

   predicted_body_mass_g                           top_feature_attributions  \
0            4681.782896  [{'feature': 'species', 'attribution': 5830.92...   
1            3875.224470  [{'feature': 'species', 'attribution': 4843.94...   
2            3303.096891  [{'feature': 'species', 'attribution': 4843.94...   
3            3976.529009  [{'feature': 'species', 'attribution': 4843.94...   
4            3457.923587  [{'feature': 'species', 'attribution': 4843.94...   

   baseline_prediction_value  prediction_value  approximation_error  \
0               -2324.072313       4681.782896                  0.0   
1               -2324.072313       3875.224470                  0.0   
2               -2324.072313       3303.096891                  0.0   
3               -2324.072313       3976.529009                  0.0   
4               -2324.072313       3457.923587                  0.0   

                               species  island  culmen_length_mm  \
0    Gentoo penguin (Pygosceli

In [25]:
df = df[['top_feature_attributions', 'baseline_prediction_value', 'prediction_value']]
pd.set_option('display.max_colwidth', None)#取消自動截斷
print(df.head())



                                                                                                                                                      top_feature_attributions  \
0    [{'feature': 'species', 'attribution': 5830.92102253957}, {'feature': 'sex', 'attribution': 3567.172505124784}, {'feature': 'island', 'attribution': -2392.238318185491}]   
1   [{'feature': 'species', 'attribution': 4843.944901373308}, {'feature': 'sex', 'attribution': 3976.741108184378}, {'feature': 'island', 'attribution': -2392.238318185491}]   
2  [{'feature': 'species', 'attribution': 4843.944901373308}, {'feature': 'sex', 'attribution': 3597.4996318787407}, {'feature': 'island', 'attribution': -2392.238318185491}]   
3   [{'feature': 'species', 'attribution': 4843.944901373308}, {'feature': 'sex', 'attribution': 3976.741108184378}, {'feature': 'island', 'attribution': -2392.238318185491}]   
4  [{'feature': 'species', 'attribution': 4843.944901373308}, {'feature': 'sex', 'attribution': 3597.499631878

In [26]:
# 將每列的 attribution 陣列拆開（每個特徵貢獻變一列）
df_exploded = df.explode('top_feature_attributions').reset_index(drop=True)

# 展開 struct 欄位（如 {'feature': 'island', 'attribution': 2483.38}）
attr = pd.json_normalize(df_exploded['top_feature_attributions'])

# 合併 baseline、prediction 與 attribution 欄位
df_result = pd.concat([
    df_exploded[['baseline_prediction_value', 'prediction_value']],
    attr
], axis=1)

# 顯示結果
print(df_result.head(10))


   baseline_prediction_value  prediction_value  feature  attribution
0               -2324.072313       4681.782896  species  5830.921023
1               -2324.072313       4681.782896      sex  3567.172505
2               -2324.072313       4681.782896   island -2392.238318
3               -2324.072313       3875.224470  species  4843.944901
4               -2324.072313       3875.224470      sex  3976.741108
5               -2324.072313       3875.224470   island -2392.238318
6               -2324.072313       3303.096891  species  4843.944901
7               -2324.072313       3303.096891      sex  3597.499632
8               -2324.072313       3303.096891   island -2392.238318
9               -2324.072313       3976.529009  species  4843.944901
